## Replacing missing values by an arbitrary number

In this recipe, we will replace missing values by an arbitrary number using pandas, Scikit-learn and Feature-Engine, all open source Python libraries.

In [1]:
import pandas as pd

# to split the data sets
from sklearn.model_selection import train_test_split

# to impute missing data with sklearn
from sklearn.impute import SimpleImputer

# to impute missing data with feature-engine
from feature_engine.missing_data_imputers import ArbitraryNumberImputer

In [2]:
# load data
data = pd.read_csv('creditApprovalUCI.csv')
data.head()

A1     A2     A3 A4 A5 A6 A7    A8   A9  A10  A11 A12 A13    A14  A15  A16
0  b  30.83  0.000  u  g  w  v  1.25    t    t    1   f   g  202.0    0    1
1  a  58.67  4.460  u  g  q  h  3.04    t    t    6   f   g   43.0  560    1
2  a  24.50    NaN  u  g  q  h   NaN  NaN  NaN    0   f   g  280.0  824    1
3  b  27.83  1.540  u  g  w  v  3.75    t    t    5   t   g  100.0    3    1
4  b  20.17  5.625  u  g  w  v  1.71    t    f    0   f   s  120.0    0    1

In [3]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data.drop('A16', axis=1), data['A16'], test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((483, 15), (207, 15))

In [4]:
# find the percentage of missing data per variable

X_train.isnull().mean()

A1     0.008282
A2     0.022774
A3     0.140787
A4     0.008282
A5     0.008282
A6     0.008282
A7     0.008282
A8     0.140787
A9     0.140787
A10    0.140787
A11    0.000000
A12    0.000000
A13    0.000000
A14    0.014493
A15    0.000000
dtype: float64

## Arbitrary imputation with pandas

In [5]:
# find the maximum value per variable
X_train[['A2','A3', 'A8', 'A11']].max()

A2     76.750
A3     26.335
A8     20.000
A11    67.000
dtype: float64

In [6]:
# replace NA with 99 in indicated numerical variables

for var in ['A2','A3', 'A8', 'A11']:
    
    X_train[var].fillna(99, inplace=True)
    X_test[var].fillna(99, inplace=True)

In [7]:
# check absence of missing values
X_train[['A2','A3', 'A8', 'A11']].isnull().sum()

A2     0
A3     0
A8     0
A11    0
dtype: int64

## Arbitrary imputation with Scikit-learn

In [8]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data[['A2', 'A3', 'A8', 'A11']],
    data['A16'],
    test_size=0.3,
    random_state=0)

In [9]:
# create an instance of the simple imputer
imputer = SimpleImputer(strategy='constant', fill_value=99)

# we fit the imputer to the train set
imputer.fit(X_train)

# we can look at the constant values:
imputer.statistics_

array([99., 99., 99., 99.])

In [10]:
# and now we impute the train and test set
# NOTE: the data is returned as a numpy array!!!

X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

In [11]:
# check that missing values were removed
pd.DataFrame(X_train).isnull().sum()

0    0
1    0
2    0
3    0
dtype: int64

## Arbitrary imputation imputation with feature engine

In [12]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data.drop('A16', axis=1), data['A16'], test_size=0.3, random_state=0)

In [13]:
# let's create an arbitrary value imputer

imputer = ArbitraryNumberImputer(
    arbitrary_number=99, variables=['A2','A3', 'A8', 'A11'])

imputer.fit(X_train)

ArbitraryNumberImputer(arbitrary_number=99, variables=['A2', 'A3', 'A8', 'A11'])

In [14]:
# dictionary with the mappings for each variable
imputer.arbitrary_number

99

In [15]:
# transform the data
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

C:\Users\Sole\Anaconda3\envs\feml\lib\site-packages\sklearn\utils\validation.py:933: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  "argument is ignored.", FutureWarning)


In [16]:
# check that null values were replaced
X_train[['A2','A3', 'A8', 'A11']].isnull().mean()

A2     0.0
A3     0.0
A8     0.0
A11    0.0
dtype: float64

## Arbitrary imputation imputation with Sklearn selecting features to impute

In [17]:
import pandas as pd

# to impute missing data with sklearn
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# to split the data sets
from sklearn.model_selection import train_test_split

In [18]:
# load data
data = pd.read_csv('creditApprovalUCI.csv')

# let's separate into training and testing set
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('A16', axis=1),data['A16' ], test_size=0.3, random_state=0)

In [19]:
# first we need to make a list with the numerical vars
features_arbitrary = ['A2', 'A3', 'A8', 'A11']
features_mean = ['A15']

# then we instantiate the imputer within a pipeline
arbitrary_imputer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value=99))])

mean_imputer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean'))])

# then we put the features list and the imputer in
# the column transformer
preprocessor = ColumnTransformer(transformers=[
    ('arbitrary_imputer', arbitrary_imputer, features_arbitrary),
    ('mean_imputer', mean_imputer, features_mean)
    ], remainder='passthrough')

In [20]:
# now we fit the preprocessor
preprocessor.fit(X_train)

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('arbitrary_imputer',
                                 Pipeline(memory=None,
                                          steps=[('imputer',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=99,
                                                                missing_values=nan,
                                                                strategy='constant',
                                                                verbose=0))],
                                          verbose=False),
                                 ['A2', 'A3', 'A8', 'A11']),
                                ('mean_imputer',
                                 Pipeline(memory=None,

In [21]:
# and now we impute the data
X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

In [22]:
# Note that Scikit-Learn transformers return NumPy arrays!!
X_train

array([[46.08, 3.0, 2.375, ..., 't', 'g', 396.0],
       [15.92, 2.875, 0.085, ..., 'f', 'g', 120.0],
       [36.33, 2.125, 0.085, ..., 'f', 'g', 50.0],
       ...,
       [19.58, 0.665, 1.665, ..., 'f', 'g', 220.0],
       [22.83, 2.29, 2.29, ..., 't', 'g', 140.0],
       [40.58, 3.29, 3.5, ..., 't', 's', 400.0]], dtype=object)